In [1]:
from datasets import Dataset, DatasetDict

/accounts/projects/sewonm/prasann/.conda/envs/scaling2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# for initial gemini indoc data, reformat to have format [query, positive, negative], be broken into train / test split
testdata = Dataset.from_json("../data/colbert_training/gemini_indoc_test.jsonl")
traindata = Dataset.from_json("../data/colbert_training/gemini_indoc_train.jsonl")
testdatafilt = Dataset.from_json("../data/colbert_training/gemini_indoc_test_filtered.jsonl")
traindatafilt = Dataset.from_json("../data/colbert_training/gemini_indoc_train_filtered.jsonl")

In [13]:
testdatafilt

Dataset({
    features: ['query', 'pos', 'neg', 'category', 'rephrased_query'],
    num_rows: 2993
})

In [14]:
def reformat_data(row):
    return {
        "query": row['rephrased_query'] if "rephrased_query" in row else row["query"],
        "positive": row["pos"][0],
        "negative": row["neg"][0]
    }

testdata = testdata.map(reformat_data).remove_columns(["pos", "neg", "posind"])
traindata = traindata.map(reformat_data).remove_columns(["pos", "neg", "posind"])
testdatafilt = testdatafilt.map(reformat_data).remove_columns(["pos", "neg", "category", "rephrased_query"])
traindatafilt = traindatafilt.map(reformat_data).remove_columns(["pos", "neg", "category", "rephrased_query"])

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map: 100%|██████████| 27000/27000 [00:03<00:00, 8432.27 examples/s] 


In [18]:
DatasetDict({"train": traindata, "test": testdata}).save_to_disk("../data/colbert_training/gemini_datav1")
DatasetDict({"train": traindata, "test": testdatafilt}).save_to_disk("../data/colbert_training/gemini_ntrain_ptest")
DatasetDict({"train": traindatafilt, "test": testdatafilt}).save_to_disk("../data/colbert_training/gemini_ptrain_ptest")

Saving the dataset (1/1 shards): 100%|██████████| 2993/2993 [00:00<00:00, 49833.88 examples/s]


In [28]:
assert all(type(x) == str for x in testdata["negative"])